In [1]:
import glob
import os

In [5]:
import otf2
from otf2.events import *

In [8]:
scorep_result_dir = './scorep-results'

# get all files in the directory
scorep_result_dir = './scorep-results'
scorep_result_dir = glob.glob(os.path.join(scorep_result_dir, '*'))

print(scorep_result_dir[0])

./scorep-results/bt.C.25.mpi_io_full


In [3]:
sample_otf2_file_path = scorep_result_dir[0] + '/traces.otf2'

In [67]:
def read_trace(trace_name):
    with otf2.reader.open(trace_name) as trace:
        return trace

In [69]:
sample_trace = read_trace(sample_otf2_file_path)

In [75]:
dir(sample_trace.definitions)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__otf2_wrapper__set_clock_properties',
 '__otf2_wrapper_funcptr__set_clock_properties',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_attributes',
 '_calling_context_properties',
 '_calling_contexts',
 '_callpath_parameters',
 '_callpaths',
 '_callsites',
 '_cart_coordinates',
 '_cart_dimensions',
 '_cart_topologies',
 '_comms',
 '_get',
 '_groups',
 '_interrupt_generators',
 '_io_file_properties',
 '_io_files',
 '_io_handles',
 '_io_paradigms',
 '_io_pre_created_handle_states',
 '_location_group_properties',
 '_location_groups',
 '_location_properties',
 '_locations',
 '_metric_class_recorders',
 '_metric_members',
 '_metrics',
 '_paradigm_prope

In [81]:
dir(sample_trace.definitions.regions)

['__class__',
 '__contains__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_callback_gen',
 '_default_type',
 '_definition_registry',
 '_elements_by_key',
 '_elements_by_ref',
 '_extra_callbacks',
 '_next_ref',
 '_ref',
 '_register',
 'create',
 'get']